In [75]:
#importing libraries
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor  
from keras import regularizers
from keras.constraints import Constraint
from keras.constraints import max_norm

### TEST DATA

In [85]:
#INITTIATE COLUMNS
train = pd.read_csv('prohack_train.csv')
df_na = pd.DataFrame(train.isnull().sum(axis = 0))
df_na[0] = df_na / train['y'].count()
df_na = df_na.rename(columns={0: "Missing_Rate"})
drops = df_na[df_na['Missing_Rate'] <= 0.5].reset_index()
columns_left = drops['index'].values.tolist()

#columns_left.append('Gender Inequality Index (GII)')
#columns_left.append('Interstellar Data Net users, total (% of population)')
#columns_left.append('Estimated gross galactic income per capita, female')
#columns_left.append('Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64))')

df2 = pd.read_csv('prohack_test.csv')
columns_left.remove('y')
df2 = df2[columns_left]

imp = IterativeImputer(max_iter=50, random_state=0)
exclude = ['galaxy'] 
imp.fit(df2.loc[:, df2.columns.difference(exclude)])
IterativeImputer(random_state=0)
df2.loc[:, df2.columns.difference(exclude)] = imp.transform(df2.loc[:, df2.columns.difference(exclude)])

cat_feats = ['galaxy']
test = pd.DataFrame(pd.get_dummies(df2,columns=cat_feats,drop_first=True))

#drop_list = ['galactic year_1016064.0']
#test = test.drop(drop_list,axis=1)
test_columns = test.columns.values.tolist()
test.shape

(890, 183)

### TRAIN DATA

In [77]:
#df2 = pd.read_csv('prohack_train_R_Cleaned_Clustered_4.csv',encoding='latin-1')
#df = df2.drop(['Unnamed: 0'],axis=1)
#df['Multiplied'] = (df['Old.age.dependency.ratio..old.age..65.and.older..per.100.creatures..ages.15.64..']) * (df['Interstellar.Data.Net.users..total....of.population.']) * (df['Estimated.gross.galactic.income.per.capita..female']) / (df['Gender.Inequality.Index..GII.'])
#drop_list = ['y']

df2 = pd.read_csv('prohack_train.csv')
columns_left.append('y')
df2 = df2[columns_left]


drop_galaxy = ['Andromeda XII','Andromeda XIX[60]','Andromeda XVIII[60]', 'Andromeda XXII[57]', 'Andromeda XXIV',
               'Hercules Dwarf', 'NGC 5253','Triangulum Galaxy (M33)','Tucana Dwarf'] 
df2 = df2[~df2['galaxy'].isin(drop_galaxy)]

#Anomalies
#df2 = df2[df2['existence expectancy index']>0.227]
#df2 = df2[df2['existence expectancy at birth']>34.25]
#df2 = df2[(df2['Gross income per capita']<150000) and (df2['Gross income per capita']>0) ]
#df2 = df2[df2['Expected years of education (galactic years)']<26.7]

imp = IterativeImputer(max_iter=50, random_state=0)
exclude = ['galaxy','y'] 
imp.fit(df2.loc[:, df2.columns.difference(exclude)])
IterativeImputer(random_state=0)
df2.loc[:, df2.columns.difference(exclude)] = imp.transform(df2.loc[:, df2.columns.difference(exclude)])

df = df2[df2['y'] < 0.5]
cat_feats = ['galaxy']
df = pd.DataFrame(pd.get_dummies(df,columns=cat_feats,drop_first=True))

test_columns.append('y')
df2 = df[test_columns]

In [5]:
### VIF REMOVALS
def calculate_vif_(X, thresh=100):
    cols = X.columns
    variables = np.arange(X.shape[1])
    dropped=True
    while dropped:
        dropped=False
        c = X[cols[variables]].values
        vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
            variables = np.delete(variables, maxloc)
            dropped=True

    print('Remaining variables:')
    print(X.columns[variables])
    return variables
    
calculate_vif_(df2)

dropping 'galactic year' at index: 0
dropping 'existence expectancy at birth' at index: 1
dropping 'existence expectancy index' at index: 0
dropping 'Population using at least basic drinking-water services (%)' at index: 7
dropping 'Intergalactic Development Index (IDI)' at index: 4
dropping 'Income Index' at index: 1
dropping 'Education Index' at index: 3
Remaining variables:
Index(['Gross income per capita',
       'Expected years of education (galactic years)',
       'Mean years of education (galactic years)',
       'Intergalactic Development Index (IDI), Rank',
       'Population using at least basic sanitation services (%)',
       'galaxy_Andromeda I', 'galaxy_Andromeda II', 'galaxy_Andromeda III',
       'galaxy_Andromeda IX', 'galaxy_Andromeda V',
       ...
       'galaxy_UGCA 438 (ESO 407-018)', 'galaxy_UGCA 86', 'galaxy_UGCA 92',
       'galaxy_Ursa Major I Dwarf (UMa I dSph)', 'galaxy_Ursa Major II Dwarf',
       'galaxy_Ursa Minor Dwarf', 'galaxy_Virgo I', 'galaxy_Willma

array([  3,   5,   6,   9,  11,  12,  13,  14,  15,  16,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183])

In [78]:
#VIF_DROPS = ['galactic year','existence expectancy at birth','existence expectancy index','Population using at least basic drinking-water services (%)',
            #'Intergalactic Development Index (IDI)','Income Index','Education Index']
#df2 = df2.drop(VIF_DROPS,axis=1)
X = df2.drop('y',axis=1)
y = df2['y']
y = np.array(y) ## SEN BU KISMI SIL
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=25)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X.shape

(3660, 183)

In [111]:
import itertools
from keras import backend as K
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

batch_list = [50,100] 
#quantile_list = [5,10] qn,quantile_list,
first_layer_list = [1,2]
second_layer_list = [1,2] 
dropout_input = [0.1,0.2]
dropout_hidden_1 = [0.1,0.2]
dropout_hidden_2 = [0.1]
patience_list = [15]
train_error = []
test_error = []
batch = []
first_layer = []
second_layer = []
third_layer = []
quantile = []
drop_input = []
drop_hidden1 = []
drop_hidden2 = []
patience_bracket = []

for  fl,sl,aa,drop_i,drop_h1,droph2,pt in itertools.product(first_layer_list, 
    second_layer_list,batch_list,dropout_input,dropout_hidden_1,dropout_hidden_2,patience_list):
    
                K.clear_session()
                #qt = QuantileTransformer(n_quantiles=qn)
                #X_train = qt.fit_transform(X_train)
                #X_test = qt.fit_transform(X_test)
                
                early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 0, patience = pt)

                # Create model and fit data to it
                model = Sequential()

                # Input Layer
                model.add(Dense(X.shape[1], input_shape=(X.shape[1],), activation = 'relu'))
                model.add(Dropout(drop_i))

                # Hidden Layers
                model.add(Dense(X.shape[1]/fl, activation = 'relu'))
                model.add(Dropout(drop_h1))

                model.add(Dense(X.shape[1]/sl, activation = 'softplus'))
                model.add(Dropout(droph2))
                
                # Output Layer
                model.add(Dense(1, activation = 'softplus'))

                model.compile(optimizer = 'adam', loss = 'mse',learning_rate=0.0001)

                model.fit(X_train,y_train,epochs = 1000, validation_data = (X_test, y_test), callbacks=[early_stop],
                          batch_size=aa,verbose = 0)

                pred_train = model.predict(X_train)
                pred_test = model.predict(X_test)

                train_error1 = np.sqrt(mean_squared_error(y_train, pred_train))
                test_error1 = np.sqrt(mean_squared_error(y_test, pred_test))

                batch.append(aa)
                test_error.append(test_error1)
                train_error.append(train_error1)
                first_layer.append(fl)
                second_layer.append(sl)
                #third_layer.append(tl)
                #quantile_list.append(qn)
                drop_input.append(drop_i)
                drop_hidden1.append(drop_h1)
                drop_hidden2.append(droph2)
                patience_bracket.append(pt)
                print(pd.DataFrame(test_error,columns=['best test_error']).sort_values(by='best test_error').min().round(4).to_string())
                print(pd.DataFrame(test_error,columns=['Median test error']).sort_values(by='Median test error').median().round(4).to_string())
                print((pd.DataFrame(test_error,columns=['Loop Finished %']).sort_values(by='Loop Finished %').count() / 32).round(2).to_string())
                print()
                
                K.clear_session()
                
                if test_error1 == min(test_error):
                    model.save("Prohack_Model_With" + str(test_error1))
                else:
                    None
   
                del model


best test_error    0.0061
Median test error    0.0061
Loop Finished %    0.03

best test_error    0.0061
Median test error    0.0066
Loop Finished %    0.06

best test_error    0.0061
Median test error    0.0072
Loop Finished %    0.09

best test_error    0.0058
Median test error    0.0066
Loop Finished %    0.12

best test_error    0.0058
Median test error    0.0072
Loop Finished %    0.16

best test_error    0.0058
Median test error    0.0072
Loop Finished %    0.19

best test_error    0.0058
Median test error    0.0072
Loop Finished %    0.22

best test_error    0.0058
Median test error    0.0073
Loop Finished %    0.25

best test_error    0.0058
Median test error    0.0075
Loop Finished %    0.28

best test_error    0.0058
Median test error    0.0075
Loop Finished %    0.31

best test_error    0.0058
Median test error    0.0075
Loop Finished %    0.34

best test_error    0.0058
Median test error    0.0076
Loop Finished %    0.38

best test_error    0.0058
Median test error    0.007

In [112]:
# After completing the entire loop, assess performance results
b = pd.DataFrame(batch,columns=['batch size'])
e = pd.DataFrame(test_error,columns=['test_error'])
t = pd.DataFrame(train_error,columns=['train_error'])
fl = pd.DataFrame(first_layer,columns=['first layer divisor'])
sl = pd.DataFrame(second_layer,columns=['second layer divisor'])
tl = pd.DataFrame(third_layer,columns=['third layer divisor'])
#qn = pd.DataFrame(quantile_list,columns=['Scaling Quantile'])
di = pd.DataFrame(drop_input,columns=['Input Dropout'])
dh1 = pd.DataFrame(drop_hidden1,columns=['Hidden 1 Dropout'])
dh2 = pd.DataFrame(drop_hidden2,columns=['Hidden 2 Dropout'])
pt = pd.DataFrame(patience_bracket,columns=['Patience'])

results = pd.concat([b,fl,sl,di,dh1,dh2,pt,t,e],axis=1)
#results.to_csv('grid_search_5.csv')
results.sort_values(by='test_error').head(3)

,batch size,first layer divisor,second layer divisor,Input Dropout,Hidden 1 Dropout,Hidden 2 Dropout,Patience,train_error,test_error
3,50,1,1,0.2,0.2,0.1,15,0.004172,0.005790
0,50,1,1,0.1,0.1,0.1,15,0.004172,0.006114
19,50,2,1,0.2,0.2,0.1,15,0.004374,0.006412


### REAL PREDICTION

In [113]:
df_pred_transform = scaler.transform(test)
preds = np.array(df_pred_transform, dtype=np.float)

In [114]:
# Model load 
from tensorflow.keras.models import load_model
model_00496 = load_model('Prohack_Model_With0.00496748783373051') 
result = model_00496.predict(preds)
result = pd.DataFrame(result)

### ALLOCATION

In [115]:
index = result
pot_inc = -np.log(index+0.01)+3

In [116]:
p2= pot_inc**2

In [117]:
result.reset_index()

ss = pd.DataFrame({
    'Index':result.index,
    'pred': result[0],
    'opt_pred':0,
    'eei':test['existence expectancy index'], # So that we can split into low and high EEI galaxies
})

In [118]:
p2.columns = ['index']
ss.loc[p2.nlargest(400,'index').index, 'opt_pred']=100
ss=ss.sort_values('pred')
ss.iloc[400:600].opt_pred = 50
ss=ss.sort_index()

C:\Users\Public\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [119]:
increase = (ss['opt_pred']*p2['index'])/1000

In [120]:
print(sum(increase), ss.loc[ss.eei < 0.7, 'opt_pred'].sum(), ss['opt_pred'].sum())

1855.6131571769688 6400 50000


In [121]:
ss[['Index', 'pred', 'opt_pred']].to_csv('submission_6_21_1.csv', index=False)

### 2ND VERSION

In [123]:
cols = ['Index','eei','pred']
test_sub = ss[cols]

In [124]:
def increase_coef(y):
    potential = 3 - np.log(0.01 + y)
    return (potential**2)/1000

In [125]:
test_sub['likely_increase'] = increase_coef(test_sub.pred)
test_sub.head()

,Index,eei,pred,likely_increase
0,0,0.456086,0.040942,0.035725
1,1,0.529835,0.036014,0.036952
2,2,0.560976,0.035949,0.036969
3,3,0.565910,0.038315,0.036361
4,4,0.588274,0.023475,0.040921


In [126]:
from scipy.stats import norm
## rank error estimation
abs_rank_error = int(0.14*len(test))
print("Rank error estimation:", abs_rank_error)

## norm distribution
opt_pred = [100*norm.cdf((499 - x)/abs_rank_error) for x in range(len(test))]

Rank error estimation: 124


In [127]:
opt_dist = test_sub[['Index', 'likely_increase']].set_index('Index')\
    .sort_values('likely_increase', ascending=False)
opt_dist['opt_pred'] = opt_pred
test_sub['opt_pred'] = test_sub['Index'].map(opt_dist.opt_pred)

test_sub.loc[test_sub.eei < 0.7, 'opt_pred'].sum(), test_sub.opt_pred.sum()

(6291.296801840401, 49947.3079854035)

In [128]:
sum(test_sub['opt_pred'] * test_sub['likely_increase'])

1848.114288402057

In [129]:
test_sub = test_sub.sort_values(by='Index').reset_index(drop=True)
test_sub[['Index', 'pred', 'opt_pred']].head()

,Index,pred,opt_pred
0,0,0.040942,90.834710
1,1,0.036014,98.434953
2,2,0.035949,98.527535
3,3,0.038315,96.063173
4,4,0.023475,99.980018


In [130]:
test_sub[['Index', 'pred', 'opt_pred']].to_csv('submission_6_21_1.csv', index=False)